In [ ]:
# Testing masked word predictions and linguistic analysis

In [4]:
import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer

In [14]:
# Gives the 5-top results for masking for test sentences
# Groups changes by noun and case changes, and verb conjugation changes. 
# Given a greater testing set, more granular linguistic features can be added to "features"

def MLMTesting(model_path, test_sentences, features, tokenizer=None):
    model = AutoModelForMaskedLM.from_pretrained(model_path)
    model.eval()
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    results = {}
    
    for property_name, sentences in test_sentences.items():
        if property_name not in features:
            continue
            
        property_results = []

        #split the sentence into words, index into the sentence and the masked word
        for sent_data in sentences:
            sentence = sent_data["sentence"]
            masked_word_index = sent_data["masked_index"]
            expected_word = sent_data["expected_word"]
            
            words = sentence.split()
            original_word = words[masked_word_index]
            words[masked_word_index] = tokenizer.mask_token
            masked_sentence = " ".join(words)
            
            inputs = tokenizer(masked_sentence, return_tensors="pt")
            inputs = {k: v.to(device) for k, v in inputs.items()}
            
            mask_token_index = torch.where(inputs["input_ids"][0] == tokenizer.mask_token_id)[0].item()
            
            with torch.no_grad():
                outputs = model(**inputs)
                predictions = outputs.logits

            #get predicted token
            predicted_token_id = torch.argmax(predictions[0, mask_token_index]).item()
            predicted_token = tokenizer.decode([predicted_token_id])
            
            is_correct = predicted_token.strip() == expected_word
            
            # return top 5 predictions
            probs = torch.nn.functional.softmax(predictions[0, mask_token_index], dim=-1)
            top_5_probs, top_5_indices = torch.topk(probs, 5)
            top_5_tokens = [tokenizer.decode([idx.item()]) for idx in top_5_indices]
            top_5_probs = top_5_probs.tolist()
            
            result = {
                "sentence": sentence,
                "masked_sentence": masked_sentence,
                "original_word": original_word,
                "expected_word": expected_word,
                "predicted_word": predicted_token,
                "is_correct": is_correct,
                "top_5_predictions": list(zip(top_5_tokens, top_5_probs))
            }
            
            property_results.append(result)
        results[property_name] = property_results
        
    
    return results
    
test_sentences = {
    "case_marking": [
        {
            "sentence": "Trakū saluos pėlės bova vėina ėš Lietovuos dėdliujū konėgaikštiu rezėdėncėju.", 
            "masked_index": 6,  
            "expected_word": "Lietovuos"
        },
        {
            "sentence": "Trakų salose pilis buvo viena iš Lietuvos didžiųjų kunigaikščių rezidencijų.",
            "masked_index": 6, 
            "expected_word": "Lietuvos"
        }, 
        {
            "sentence": "Dabartėis lėteratūrėnės kalbuos pagrinds sodarīts palē vakarū aukštātiu pėitietiu tarmė.", 
            "masked_index": 2,  
            "expected_word": "kalbuos"
        },
        {
            "sentence": "Dabartinės literatūrinės kalbos pagrindas sudarytas pagal vakarų aukštaičių pietinę tarmę.",
            "masked_index": 2, 
            "expected_word": "kalbos"
        },
        {
            "sentence": "Senovės istorijoje žemėlapiuose sienos net labai skyrėsi.",
            "masked_index": 4, 
            "expected_word": "labai"
        },
        {
            "sentence": "Žemaitē gīven vėsam svietė, kor īr lietoviu, nes oficelē anodom neskėramė vėinė nu kėtū.",
            "masked_index": 5, 
            "expected_word": "īr"
        },
    ],
    "verb_conjugation": [
        {
            "sentence": "Sena meilė nerūdie, laikou einou viel atgīn.", 
            "masked_index": 4,  
            "expected_word": "einou"
        },
        {
            "sentence": "Sena meilė nerūdija, laikas vėl ateina atgal.", 
            "masked_index": 5, 
            "expected_word": "ateina"
        },
        {
            "sentence": "Vėišuoji īstaiga skėrta tīrinietė, skaitmenėzoutė ė kėtēp nauduotė fololuora.", 
            "masked_index": 1, 
            "expected_word": "īstaiga"
        },
        {
            "sentence": "Viešoji institucija skirta tirti, skaitmenizuoti ir kitaip turtinti žemaičių kalbą.", 
            "masked_index": 2, 
            "expected_word": "skirta"
        },  
    ]
}
features = ["case_marking", "verb_conjugation"]

tokenizer = AutoTokenizer.from_pretrained("EMBEDDIA/litlat-bert")

probe_results = MLMTesting(
    "./samogitian_litlat_bert2",
    test_sentences,
    features,
    tokenizer
)
probe_results

{'case_marking': [{'sentence': 'Trakū saluos pėlės bova vėina ėš Lietovuos dėdliujū konėgaikštiu rezėdėncėju.',
   'masked_sentence': 'Trakū saluos pėlės bova vėina ėš <mask> dėdliujū konėgaikštiu rezėdėncėju.',
   'original_word': 'Lietovuos',
   'expected_word': 'Lietovuos',
   'predicted_word': '3',
   'is_correct': False,
   'top_5_predictions': [('3', 0.2426135540008545),
    ('LDK', 0.1298864483833313),
    ('4', 0.06336399167776108),
    ('5', 0.037293028086423874),
    ('2', 0.03159737214446068)]},
  {'sentence': 'Trakų salose pilis buvo viena iš Lietuvos didžiųjų kunigaikščių rezidencijų.',
   'masked_sentence': 'Trakų salose pilis buvo viena iš <mask> didžiųjų kunigaikščių rezidencijų.',
   'original_word': 'Lietuvos',
   'expected_word': 'Lietuvos',
   'predicted_word': 'Lietuvos',
   'is_correct': True,
   'top_5_predictions': [('Lietuvos', 0.95196133852005),
    ('LDK', 0.03640574589371681),
    ('didžiųjų', 0.003857015399262309),
    ('Lenkijos', 0.0010549481958150864),
 